<img src="../../contrib/img/logo.png" width=150 align="left" /> <img src="../../contrib/img/logo2.jpg" width=450 align="right" />


# <center><font color= #1e8449 > <b>MAPS</font></center>

> # <font color='steelblue'> <b>FLOWS ON MAP</font>

### <font color='steelblue'>Description</font>

Realizar mapa que represente flujos de movimientos.
 
- Los datos para el mapa vendrán de un fichero generalmente csv que se cargará a un dataframe
- La idea es visualizar flujos de elementos (movimientos de individuos, coches, etc)
- Estos datos traerán la información de los puntos de origen y destino (Lat, Long) así como la asociada a cada elemento para ser visualizada
  
- Se aporta imagen a modo ejemplo de lo que se desea
- Se debe poder seleccionar qué info mostrar, en un rango de fechas y de horas
    - En código se indicará la columna de fechas
    - En código se indicará la columna de horas
    - En el mapa se podrá elegir los rangos entre los max y mínimos de esas columnas
                Fechas: 10/10/2020 - 10/10/2020
                Horas:  08:00:00 - 14:30:00

- Existirá una leyenda que mostrará la info de movimientos totales en base los movimientos seleccionados

- Se adjunta set de datos de MOVIMIENTOS para prueba

Se recomienda ver este ejemplo
https://plotly.com/python/lines-on-maps/  (projection="orthographic")


<img src="../../contrib/img/FLUJOS.jpg" width=550 align="left" />

## <font color='green'>Settings</font>

In [1]:
# Libraries to use
import geopandas as gpd
import pandas as pd
import plotly.graph_objects as go
from jupyter_dash import JupyterDash
from datetime import date
from dash import Dash, dcc, html
from dash.dependencies import Input, Output
import matplotlib.pyplot as plt
import numpy as np
import os

## <font color='green'>Data Load</font>

<font color='green'><b> MOVIMIENTOS</b></font>

In [2]:
MOVIMIENTOS = pd.read_csv("../../contrib/data/MOVIMIENTOS.CSV")

In [3]:
MOVIMIENTOS

,Id_individuo,fecha_origen,hora_origen,lat_origen,long_origen,hora_destino,fecha_destino,lat_destino,long_destino
0,0f4bed44533de213f694d7d4cd40c0f594c055b8053890...,2020-05-18,13:00:00,40.408368,-3.693346,15:30:23,2020-05-18,40.446367,-3.703667
1,d829167ca2cd17d79feb303acc1b9ed8a2f43a62d85f8a...,2020-05-18,13:00:00,40.403250,-3.672602,15:51:30,2020-05-18,40.438532,-3.698221
2,c20228e7bec9c768c7ca7b3bfad323b01fc45b19bd0e7b...,2020-05-18,13:00:00,40.441750,-3.714472,13:08:54,2020-05-18,40.397262,-3.694503
3,61149ceeae72bdea701f927b0b8dd1b47bbdd7cd7ad930...,2020-05-18,14:00:00,40.466361,-3.688639,14:02:34,2020-05-18,40.457282,-3.700967
4,6d9627a3cec2eeb7d176314ab755b8f54560bd1b22106d...,2020-05-18,14:00:00,40.426948,-3.703592,14:05:44,2020-05-18,40.421501,-3.680008
...,...,...,...,...,...,...,...,...,...
99995,2780ba50d2de5f7ddeb18527f5d002cfec59b52ced9482...,2021-01-30,12:00:00,40.420589,-3.705842,12:18:46,2021-01-30,40.403250,-3.672602
99996,567c0bee5573257f0fd9ca9484177f359ec603110038ec...,2021-01-30,12:00:00,40.459235,-3.691533,12:21:59,2021-01-30,40.427657,-3.720513
99997,261bada1c3bf446755676086116bf3137dc62c7e39f1bb...,2021-01-30,12:00:00,40.459235,-3.691533,12:21:35,2021-01-30,40.427657,-3.720513
99998,ca96db995cfce1a217fbc322cb52e7660003b62c6f9cfc...,2021-01-30,12:00:00,40.430167,-3.663889,12:09:44,2021-01-30,40.420078,-3.706538


In [4]:
MOVIMIENTOS["fecha_destino"] = pd.to_datetime(MOVIMIENTOS["fecha_destino"])
MOVIMIENTOS["fecha_origen"] = pd.to_datetime(MOVIMIENTOS["fecha_origen"])
MOVIMIENTOS["hora_origen"] = pd.to_numeric(MOVIMIENTOS["hora_origen"].apply(lambda x: x[:2]))
MOVIMIENTOS["hora_destino"] = pd.to_numeric(MOVIMIENTOS["hora_destino"].apply(lambda x: x[:2]))

## <font color='green'>PARAMETERS</font>

Variables

In [5]:
# Columna para la selección de fecha
column_fecha='fecha_origen'

# Columna para la selección de hora
column_hora='hora_origen'

Parámetros para configurar el aspecto del mapa

In [6]:
# Titulo del mapa
titulo = 'Flujos'
# Tipo de mapa
estilo_mapa='open-street-map' # "open-street-map", "carto-positron", "carto-darkmatter", "stamen-terrain",                           
            # "stamen-toner" or "stamen-watercolor" 
    

## <font color='green'>FLOWS MAP</font>

In [7]:
import plotly.express as px

def plot_flows(df_interest): 
    min_lat = min(df_interest["lat_origen"].min(), df_interest["lat_destino"].min())
    max_lat = max(df_interest["lat_origen"].max(), df_interest["lat_destino"].max())
    
    min_long = min(df_interest["long_origen"].min(), df_interest["long_destino"].min())
    max_long = max(df_interest["long_origen"].max(), df_interest["long_destino"].max())
    
    lons = np.empty(3 * len(df_interest))
    lons[::3] = df_interest['long_origen']
    lons[1::3] = df_interest['long_destino']
    lons[2::3] = None
    
    lats = np.empty(3 * len(df_interest))
    lats[::3] = df_interest['lat_origen']
    lats[1::3] = df_interest['lat_destino']
    lats[2::3] = None
    
    fig = go.Figure(data=px.line_mapbox(
            lon = lons,
            lat = lats
        )
    )
    
    fig.update_layout(title_text = titulo)
    fig.update_layout(mapbox_style=estilo_mapa)
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
    return fig


def filter_df(df, from_date, to_date, from_hour, to_hour):
    if from_date is not None:
        df = df[df[column_fecha]  >= from_date]
    
    if to_date is not None:
        df = df[df[column_fecha]  <= to_date]
    
    if from_hour is not None:
        df = df[df[column_hora] >= from_hour]

    if to_hour is not None:
        df = df[df[column_hora] <= to_hour]
    
    return df

In [8]:
app = JupyterDash()

app.layout = html.Div([
    html.Div(
    [
        "Selecciona una fecha de inicio y fin",
        html.Br(),
        dcc.DatePickerRange(
            id='date-picker-range',
            min_date_allowed=MOVIMIENTOS[column_fecha].min(),
            max_date_allowed=MOVIMIENTOS[column_fecha].max(),
            initial_visible_month=MOVIMIENTOS[column_fecha].min(),
            end_date=MOVIMIENTOS[column_fecha].max()
        ),
        html.Br(),
        "Selecciona una hora de inicio y fin",
        html.Br(),
        dcc.RangeSlider(id='hour-selector', min=0, max=24, step=1, allowCross=False),
        html.Div(id="statistics")
    ]),
    html.Div([dcc.Graph(
        id='geo-map'
    )])
])

@app.callback(
    [Output('geo-map', 'figure'), Output("statistics", "children")],
    [Input('date-picker-range', 'start_date'), Input('date-picker-range', 'end_date'), Input('hour-selector', 'value')])
def update_output(start_date, end_date, hours):
        
    if hours is None:
        hours = [0, 24]

    df_interest = filter_df(MOVIMIENTOS.copy(), start_date, end_date, hours[0], hours[1])    
    statistics = f"Hay {len(df_interest)} rutas en las fechas seleccionadas de un total de {len(df_interest)}"
    return plot_flows(df_interest), statistics

app.run_server(debug=True)

Dash is running on http://127.0.0.1:8050/

Dash app running on http://127.0.0.1:8050/
